In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import json

# Read recipe inputs
disk_usage = dataiku.Dataset("disk_usage")
df = disk_usage.get_dataframe()

In [0]:
df.info()

In [0]:
df

In [0]:
for key, value in df["data"].items():
    print(value)
    dict = json.loads(value)
dict

In [0]:
df = pd.json_normalize(dict["result"])
df = df.explode('values')
df[['Timestamp','CPU_Usage']] = pd.DataFrame(df['values'].tolist(),index=df.index)
df

In [0]:
df.rename(columns={'metric.hostname': 'Broker', 'metric.dc': 'Data_Center','metric.mountpoint':'Disk'}, inplace=True)
df['Broker_Disk'] = df['Broker'] + df['Disk']
df = df.drop(['metric.cluster', 'metric.job','metric.device','metric.fstype','values','Broker','Disk'], axis=1)
df

In [0]:
list(df['Broker_Disk'].unique())

In [0]:
dfx = pd.pivot_table ( df, index=['Timestamp'], columns = df.groupby(['Timestamp']).cumcount().add(1), values = ['CPU_Usage'], aggfunc = 'sum')
dfx.columns = list(df['Broker_Disk'].unique())
dfx = dfx.reset_index()
dfx

In [0]:
df = dfx

In [0]:
# Write recipe outputs
json = dataiku.Dataset("Json")
json.write_with_schema(df)